# Introduction #

Run the cell below to set everything up.

In [1]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.computer_vision.ex6 import *

# Imports
import visiontools
from visiontools import StanfordCars
import tensorflow as tf
import tensorflow_datasets as tfds

# Load training and validation sets
DATA_DIR = '/kaggle/input/stanford-cars-for-learn/'
(ds_train_, ds_valid_), ds_info = tfds.load(
    'stanford_cars/simple_2',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
    data_dir=DATA_DIR,
    download=False,
)

# Create data pipeline
BATCH_SIZE = 32
AUTO = tf.data.experimental.AUTOTUNE
SIZE = [192, 192]

preprocess = visiontools.make_preprocessor(SIZE)

ds_train = (ds_train_
            .map(preprocess, AUTO)
            .cache()
#            .shuffle(1000)
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

ds_valid = (ds_valid_
            .map(preprocess, AUTO)
            .cache()
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

Run the cell below if you'd like to see a few examples of the new dataset.

In [ ]:
tfds.show_examples(ds_train_, ds_info)

# Fine Tune a Model

This time you'll use the custom convnet you made in Lesson 2 as the base model.

### 1) Define Model

Here is a diagram for the model you'll define:

<!--TODO: ex6.1 model-->

In [119]:
import tensorflow.keras.layers as layers
from tensorflow.keras import Sequential

NUM_CLASSES = ds_info.features['label'].num_classes

pretrained_base = tf.keras.models.load_model('mini_vgg_headless.h5')
pretrained_base.trainable = False

model = Sequential([
    # YOUR CODE HERE
    ----
])
q_1.check()

You can see the model you defined by running the following cell, if you'd like to compare.

In [123]:
model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_4 (Model)              (None, 12, 12, 512)       6860672   
_________________________________________________________________
flatten_25 (Flatten)         (None, 73728)             0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 73728)             0         
_________________________________________________________________
dense_60 (Dense)             (None, 16)                1179664   
_________________________________________________________________
dense_61 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_62 (Dense)             (None, 1)                 17        
Total params: 8,040,625
Trainable params: 1,179,953
Non-trainable params: 6,860,672
___________________________________

In [116]:
import tensorflow.keras.layers as layers
from tensorflow.keras import Sequential

NUM_CLASSES = ds_info.features['label'].num_classes

pretrained_base = tf.keras.models.load_model('mini_vgg_headless.h5')
pretrained_base.trainable = False

model = Sequential([
    pretrained_base,
    layers.Flatten(),
    layers.Dense(4, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_4 (Model)              (None, 12, 12, 512)       6860672   
_________________________________________________________________
flatten_23 (Flatten)         (None, 73728)             0         
_________________________________________________________________
dense_54 (Dense)             (None, 4)                 294916    
_________________________________________________________________
dense_55 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_56 (Dense)             (None, 1)                 5         
Total params: 7,155,613
Trainable params: 294,941
Non-trainable params: 6,860,672
_________________________________________________________________


### 2) Train Head

Compile the model with the following parameters:
- 10 epochs
- `'binary_crossentropy'` loss
- `'AUC'` metric

In [134]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.002,
    decay_steps=41,
    decay_rate=0.9,
    staircase=True,
)
optimizer = tf.keras.optimizers.Adam(lr_schedule)

# Number of epochs per round of training
# YOUR CODE HERE
EPOCHS = ____

model.compile(
    optimizer=optimizer,
    # YOUR CODE HERE
    ____
)
q_2.check()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 6, 6, 512)         14714688  
_________________________________________________________________
flatten_26 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 18432)             0         
_________________________________________________________________
dense_63 (Dense)             (None, 4)                 73732     
_________________________________________________________________
dense_64 (Dense)             (None, 1)                 5         
Total params: 14,788,425
Trainable params: 73,737
Non-trainable params: 14,714,688
_________________________________________________________________


In [135]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.002,
    decay_steps=41,
    decay_rate=0.9,
    staircase=True,
)
optimizer = tf.keras.optimizers.Adam(lr_schedule)

# Number of epochs per round of training
EPOCHS = 10

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['AUC'],
)

Once you've got the right answer, run the cell below to train your model.

In [ ]:
history = model.fit(ds_train,
                    validation_data=ds_valid,
                    epochs=EPOCHS)

Epoch 1/10
41/41 [==============================] - 12s 289ms/step - loss: 0.7495 - binary_accuracy: 0.7548 - AUC: 0.4809 - val_loss: 0.6753 - val_binary_accuracy: 0.8035 - val_AUC: 0.5000
Epoch 2/10
41/41 [==============================] - 10s 241ms/step - loss: 0.6650 - binary_accuracy: 0.8037 - AUC: 0.5171 - val_loss: 0.6557 - val_binary_accuracy: 0.8035 - val_AUC: 0.5000
Epoch 3/10
35/41 [========================>.....] - ETA: 0s - loss: 0.6479 - binary_accuracy: 0.8080 - AUC: 0.5359

### 3) Fine Tune Base

Let's fine tune the convolutional layers in the last block of the model. Run the cell below to get a list of their indices.

In [96]:
for idx, layer in enumerate(pretrained_base.layers):
    print(idx, layer.name)

0 input_7
1 block1_conv1
2 block1_pool
3 block2_conv1
4 block2_pool
5 block3_conv1
6 block3_conv2
7 block3_pool
8 block4_conv1
9 block4_conv2
10 block4_conv3
11 block4_pool


What are the indices you'll select for retraining?


In [131]:
# YOUR CODE HERE
idx_tuned=[____]

q_3.check()

Unfreezing layer at index 10.
Epoch 61/70
41/41 [==============================] - 6s 155ms/step - loss: 0.5652 - binary_accuracy: 0.8037 - AUC: 0.6415 - val_loss: 0.5926 - val_binary_accuracy: 0.8035 - val_AUC: 0.6900
Epoch 62/70
41/41 [==============================] - 5s 131ms/step - loss: 0.5713 - binary_accuracy: 0.8037 - AUC: 0.6880 - val_loss: 0.5859 - val_binary_accuracy: 0.8035 - val_AUC: 0.6909
Epoch 63/70
41/41 [==============================] - 5s 131ms/step - loss: 0.5415 - binary_accuracy: 0.8037 - AUC: 0.6796 - val_loss: 0.5467 - val_binary_accuracy: 0.8035 - val_AUC: 0.6838
Epoch 64/70
41/41 [==============================] - 5s 130ms/step - loss: 0.5185 - binary_accuracy: 0.8037 - AUC: 0.6772 - val_loss: 0.5235 - val_binary_accuracy: 0.8035 - val_AUC: 0.6901
Epoch 65/70
41/41 [==============================] - 5s 132ms/step - loss: 0.5159 - binary_accuracy: 0.8037 - AUC: 0.6681 - val_loss: 0.5208 - val_binary_accuracy: 0.8035 - val_AUC: 0.7005
Epoch 66/70
41/41 [======

KeyboardInterrupt: 

Once you've got the right answer, run the cell below to start the training loop.

In [ ]:
for r, idx in enumerate(idx_tuned):
    print("Unfreezing layer at index {}.".format(idx))
    pretrained_base.layers[idx].trainable = True

    # Recompile model after unfreezing a layer.
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['AUC'],
    )
    # Define epochs
    INIT, = history.epoch[-1] + 1, # start after last iteration's end
    TOTAL = history.epoch[-1] + 1 + EPOCHS
    # Retrain with layer at idx unfrozen.
    history = model.fit(
        ds_train,
        validation_data=ds_valid,
        initial_epoch=INIT,
        epochs=TOTAL,
    )
    # Concatenate this round's history to previous history
    history_frame = pd.concat([history_frame, pd.DataFrame(history.history)])

### 4) Evaluate

Run the cell below to see the training curves.

In [ ]:
import pandas as pd

history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['AUC', 'val_AUC']].plot();

What do you think?

In [ ]:
q_4.solution()

# Conclusion #

The technique we saw in this lesson is just one way to do transfer learning. There are several others which might be better in other situations.

- save "bottlenecks"
- frozen base (like in Lesson 1)
- fine tuning entire base with warmup

We encourage you to explore these on your own!

The technique we'll look at in the next lesson -- **data augmentation** -- is a great complement to fine tuning.